In [1]:
import os,sys
sys.path.append("..")
import numpy as np
import pandas as pd
import ibmseti
import collections
import scipy.io
import matplotlib.pyplot as plt
import commonutils as cu
import PIL
from PIL import Image

Using TensorFlow backend.


In [2]:
import keras
from keras import backend as K
K.set_image_dim_ordering('tf')
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

In [3]:
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from model_specs import vgg_fine_tune

In [4]:
# Might want to tune these parameters
train_datagen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,
                                   rotation_range=0, width_shift_range=0.2,
                                   height_shift_range=0.1,zoom_range=0.1,
                                   horizontal_flip=True, vertical_flip = True, fill_mode='constant')
test_datagen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True)

# # Create a sample dataset array for fitting whitening / normalization parameters
# #(linux command for putting all images in one place) 
# # find train -name '*.jpg' -type f -exec cp {} . \;
trainImgDir = '../data/imagesDataset_512x256_8/train_one_folder/'
files = [f for f in os.listdir(trainImgDir) if f.endswith('.jpg')]
size = img_to_array(load_img(os.path.join(trainImgDir,files[0]),grayscale=False)).shape

In [5]:
print size

(256, 512, 3)


In [6]:
model = vgg_fine_tune.build(input_shape= (size[0],size[1],size[2]),nb_classes=4)

model_specs.py:33: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1024, kernel_initializer="he_normal", activation="relu", input_shape=(65536,))`
  activation='relu',init="he_normal"))
model_specs.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, kernel_initializer="he_normal", activation="relu")`
  model.add(Dense(256,activation='relu',init="he_normal"))
model_specs.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, kernel_initializer="he_normal", activation="relu")`
  model.add(Dense(256,activation='relu',init="he_normal"))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 512, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 512, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 512, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 256, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 256, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 256, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 128, 128)      0         
__________

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])
# model.fit(train_data, train_labels,
#           epochs=50,
#           batch_size=batch_size,
#           validation_data=(validation_data, validation_labels))

In [8]:
trainDataArray = np.zeros((len(files)//5,)+size) # Only 1/10 the data, because memory constraints
for i in range(len(files)//5):
    trainDataArray[i] = img_to_array(load_img(os.path.join(trainImgDir,files[i])))
    
train_datagen.fit(trainDataArray)
test_datagen.fit(trainDataArray)

datasetPath = "../data/imagesDataset_512x256_8/train/"
valDataPath = "../data/imagesDataset_512x256_8/validation/"
classList = ['0-noise','2-narrowband','3-narrowbanddrd','5-squiggle']
batch_size = 8
train_generator = train_datagen.flow_from_directory(directory=datasetPath,batch_size=batch_size,class_mode='categorical',
                            classes=classList,target_size=(256,512))
validation_generator = test_datagen.flow_from_directory(directory=valDataPath,batch_size=batch_size,
        class_mode='categorical',classes=classList,target_size=(256,512))

Found 7584 images belonging to 4 classes.
Found 1498 images belonging to 4 classes.


In [ ]:
epochs = 50
nb_train_samples = 4000
nb_validation_samples = 1000
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    verbose=1)


Epoch 1/50
500/500 [==============================] - 335s - loss: 1.8596 - acc: 0.3307 - val_loss: 1.1669 - val_acc: 0.4360
Epoch 2/50
500/500 [==============================] - 328s - loss: 1.3996 - acc: 0.3815 - val_loss: 1.1071 - val_acc: 0.5020
Epoch 3/50
500/500 [==============================] - 327s - loss: 1.3760 - acc: 0.3987 - val_loss: 1.2119 - val_acc: 0.3541
Epoch 4/50
500/500 [==============================] - 327s - loss: 1.3386 - acc: 0.3960 - val_loss: 1.0628 - val_acc: 0.4930
Epoch 5/50
500/500 [==============================] - 327s - loss: 1.2937 - acc: 0.4320 - val_loss: 1.0119 - val_acc: 0.5231
Epoch 6/50
500/500 [==============================] - 327s - loss: 1.2951 - acc: 0.4195 - val_loss: 1.2124 - val_acc: 0.5101
Epoch 7/50
500/500 [==============================] - 327s - loss: 1.2938 - acc: 0.4300 - val_loss: 1.3037 - val_acc: 0.4650
Epoch 8/50
500/500 [==============================] - 327s - loss: 1.2588 - acc: 0.4460 - val_loss: 0.9948 - val_acc: 0.5670


In [ ]:
model.predict_generator(validation_generator,verbose=1,validation_steps=nb_validation_samples// batch_size)

In [ ]:
# dimensions of our images.

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'w'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'w'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy'))
    train_labels = np.array(
        [0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))

    validation_data = np.load(open('bottleneck_features_validation.npy'))
    validation_labels = np.array(
        [0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)